In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd gdrive/My\ Drive/PSOSM/Homework_3

/content/gdrive/My Drive/PSOSM/Homework_3


In [0]:
import matplotlib.pyplot as plt
from datetime import datetime
import datetime as dt
import numpy as np
import pandas as pd
import operator
import json

In [0]:
url = 'https://raw.githubusercontent.com/Ashwin-19/HW3/master/Tweets.csv?token=AJD3RIUGFMHVMMULM6O7GJ26POKIC'
dataset = pd.read_csv(url, error_bad_lines=False)

In [0]:
# removing null columns

dataset = dataset.drop(labels='Unnamed: 0',axis=1)

for column in list(dataset.columns):
  if dataset[column].isnull().values.all(axis=0):
    dataset = dataset.drop(labels=column,axis=1)

In [6]:
!pip3 install twint

     |████████████████████████████████| 1.2MB 12.4MB/s 
     |████████████████████████████████| 245kB 49.3MB/s 
     |████████████████████████████████| 92kB 14.4MB/s 
     |████████████████████████████████| 153kB 49.0MB/s 
     |████████████████████████████████| 256kB 49.2MB/s 
     |████████████████████████████████| 235kB 41.1MB/s 
  Created wheel for twint: filename=twint-2.1.15-cp36-none-any.whl size=33270 sha256=584065d95194959e7ca37eb681e24891f10498ae34a468110fb0afb68b423882
  Stored in directory: /root/.cache/pip/wheels/5a/c7/4f/2c8719f481035bd3c4d9b296bc0378bc21059503f4051b1df8
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13484 sha256=73243922609362b4ea5d7009dd60004bef0e49d224c8f41f518ddb62b6855198
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for googletransx: filename=googletransx-2.4.2-cp36-none-any.whl size=15969 sha256=0f1402e1875d20311a543418da2936c0f3f

In [7]:
# initialize twint
import twint
client = twint.Config()
client.Store_object = True

# retrieve user specific features: followers, following, bio, verified, account age
UserData = []
count = 0
for index, row in dataset.iterrows():
  client.Username = row['username']
  twint.run.Lookup(client)
  try:
    UserData.append(twint.output.users_list[0])
  except Exception:
    UserData.append(-1)
  twint.output.clean_lists()

1057575335280046080 | Khulasa News | @news_khulasa | Private: 0 | Verified: 0 | Bio: Official Twitter account of khulasa-news. We provide you all the news from the country and from the world as well. | Location:  | Url: https://khulasa-news.com/ | Joined: 31 Oct 2018 3:09 AM | Tweets: 2665 | Following: 1491 | Followers: 1517 | Likes: 673 | Media: 192 | Avatar: https://pbs.twimg.com/profile_images/1062227898591920128/tVPXptjZ_400x400.jpg
215900941 | annu kumar singh | @annuu143 | Private: 0 | Verified: 0 | Bio:  | Location:  | Url:  | Joined: 14 Nov 2010 10:50 PM | Tweets: 2217 | Following: 55 | Followers: 10 | Likes: 137 | Media: 338 | Avatar: https://abs.twimg.com/sticky/default_profile_images/default_profile_400x400.png
2264849412 | The Logical Indian | @LogicalIndians | Private: 0 | Verified: 1 | Bio: Handpicked, newsworthy stories which deserve the attention of a rational generation. | Location: India | Url: http://www.thelogicalindian.com | Joined: 27 Dec 2013 11:58 AM | Tweets: 2

CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable


251173245 | Malavika | @1dragonrose | Private: 0 | Verified: 0 | Bio: Just listening in | Location:  | Url:  | Joined: 12 Feb 2011 7:58 AM | Tweets: 123 | Following: 18 | Followers: 7 | Likes: 127 | Media: 15 | Avatar: https://pbs.twimg.com/profile_images/1069796776604024832/RcNpfd1g_400x400.jpg
779295832071737344 | Shivani Sharma | @Shivani_loves94 | Private: 0 | Verified: 0 | Bio: Works as Assistant Producer @TezTVChannel aajtak  Studied Mass Communication from ITMI | Location:  | Url:  | Joined: 23 Sep 2016 5:26 AM | Tweets: 135 | Following: 248 | Followers: 32 | Likes: 1797 | Media: 3 | Avatar: https://pbs.twimg.com/profile_images/818477844850307072/u8WyKSaP_400x400.jpg
739989998192103425 | Vilas Devkar | @vilas_devkar | Private: 0 | Verified: 0 | Bio: Mission Accomplished 2019 👍 Next mission 2024 👍👍👍  An atheist 'Bhakt'. | Location:  | Url:  | Joined: 6 Jun 2016 6:18 PM | Tweets: 1850 | Following: 83 | Followers: 90 | Likes: 196 | Media: 64 | Avatar: https://pbs.twimg.com/profile_

CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable


152325230 | Rohit Goyal | @byrohit | Private: 0 | Verified: 0 | Bio: believer of Hindi Hindu Hindustan. RSS follower, a Sanghi and everything liberals hate 😂 tweets are my personal views. | Location: New Delhi, India | Url: http://www.livemint.com | Joined: 5 Jun 2010 9:51 AM | Tweets: 1160 | Following: 419 | Followers: 206 | Likes: 18653 | Media: 38 | Avatar: https://pbs.twimg.com/profile_images/973431595171500032/hivBZe4j_400x400.jpg
895966977407074304 | EastMojo | @EastMojo | Private: 0 | Verified: 1 | Bio: Mobile-first digital news platform | New media storyteller from Northeast India | Location: Guwahati, India | Url: http://www.eastmojo.com | Joined: 11 Aug 2017 4:15 AM | Tweets: 9075 | Following: 603 | Followers: 7219 | Likes: 168 | Media: 2818 | Avatar: https://pbs.twimg.com/profile_images/1109328200209858560/KjmAdIgK_400x400.png
594981026 | Desh Bhakt | @WebWarrior10 | Private: 0 | Verified: 0 | Bio: यदा यदा हि धर्मस्य ग्लानिर्भव-ति भारत अभ्युत्थान-मधर्मस्य तदात्मानं सृजाम्यहम

...

2543144689 | aprasadi | @AmolAprasadi | Private: 0 | Verified: 0 | Bio:  | Location: Mumbai -City that never sleeps | Url:  | Joined: 3 Jun 2014 2:17 AM | Tweets: 11052 | Following: 149 | Followers: 233 | Likes: 90291 | Media: 440 | Avatar: https://pbs.twimg.com/profile_images/979622096149114880/uqw0jMGh_400x400.jpg
1114709839 | sayeed | @sayeedzafar94 | Private: 0 | Verified: 0 | Bio: Nothing can be achieved without hard work. | Location: New Delhi | Url:  | Joined: 23 Jan 2013 8:03 AM | Tweets: 561 | Following: 284 | Followers: 22 | Likes: 1896 | Media: 41 | Avatar: https://pbs.twimg.com/profile_images/1202584173799772162/xQUg9TzT_400x400.jpg
101713379 | Kamal K Mishra : कमल के मिश्र : কমল কে মিশ্র | @kamalKantmishra | Private: 0 | Verified: 0 | Bio: INDIAN | Farmer | Traveller| Observer | Believer | Student of Life | Marketeer | Vice – President & Head: Marketing & Strategy |   RTs are not endorsements. | Location: Bhagalpur, New Delhi, Kolkata  | Url:  | Joined: 4 Jan 2010 1:37 AM 

CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable


95159840 | Andy 🇮🇳 | @rudra14may | Private: 0 | Verified: 0 | Bio:  | Location: Somewhere on earth | Url:  | Joined: 7 Dec 2009 12:02 AM | Tweets: 10258 | Following: 92 | Followers: 45 | Likes: 33979 | Media: 23 | Avatar: https://pbs.twimg.com/profile_images/477565710839713792/OG_Dyx4S_400x400.jpeg
854075077314441217 | Mohammed Faiz | @MDFaiz45 | Private: 0 | Verified: 0 | Bio: Student | Social Activist | Secular Minded | Save Our Constitution | Thoughts Are Personal                  • Follow Me For Guaranteed Follow Back• | Location:  | Url:  | Joined: 17 Apr 2017 1:52 PM | Tweets: 333 | Following: 239 | Followers: 40 | Likes: 158 | Media: 23 | Avatar: https://pbs.twimg.com/profile_images/1206406447807913985/0xCrNP2h_400x400.jpg
1177858524614774790 | Kunalanand | @Kunalanand2020 | Private: 0 | Verified: 0 | Bio:  | Location:  | Url:  | Joined: 28 Sep 2019 1:12 AM | Tweets: 1466 | Following: 105 | Followers: 21 | Likes: 299 | Media: 49 | Avatar: https://pbs.twimg.com/profile_images/117

CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable


1410889154 | Mudassir Nazar | @AmnotNihilist | Private: 0 | Verified: 0 | Bio: |English Literature|International Relations| |Tea Positive| Hindustani| | Location: New Delhi, India | Url:  | Joined: 7 May 2013 11:17 AM | Tweets: 1201 | Following: 277 | Followers: 77 | Likes: 1392 | Media: 86 | Avatar: https://pbs.twimg.com/profile_images/1001202647007834113/43HcAKsS_400x400.jpg
861687457317650432 | Aftab Alam | @Aftab_tiss2015 | Private: 0 | Verified: 0 | Bio:  | Location: multiverse   | Url:  | Joined: 8 May 2017 2:01 PM | Tweets: 1780 | Following: 382 | Followers: 94 | Likes: 1627 | Media: 28 | Avatar: https://pbs.twimg.com/profile_images/861918966104588288/29WhQcnp_400x400.jpg
1642550696 | Rakesh | @TheNewIndia | Private: 0 | Verified: 0 | Bio: Proud Hindu, Medical, Arsenal Fan | Location: UK, Punjab | Url:  | Joined: 3 Aug 2013 3:22 AM | Tweets: 16032 | Following: 585 | Followers: 4718 | Likes: 11 | Media: 3190 | Avatar: https://pbs.twimg.com/profile_images/1231662193516699651/lDGdJ

In [0]:
User_df = {"followers":[],"following":[],"bio":[],"verified":[],"age":[],"user_location":[]}

for user in UserData:
  if user != -1:
    User_df['followers'].append(user.followers)
    User_df['following'].append(user.following)
    User_df['bio'].append(user.bio)
    User_df['verified'].append(user.is_verified)
    User_df['age'].append((datetime.now() - datetime.strptime(user.join_date,"%d %b %Y")).days)
    User_df['user_location'].append(user.location)
  else:
    User_df['followers'].append(np.nan)
    User_df['following'].append(np.nan)
    User_df['bio'].append(np.nan)
    User_df['verified'].append(np.nan)
    User_df['age'].append(np.nan)
    User_df['user_location'].append(np.nan)

In [0]:
dataset['followers'] = User_df['followers']
dataset['following'] = User_df['following']
dataset['bio'] = User_df['bio']
dataset['verified'] = User_df['verified']
dataset['age'] = User_df['age']
dataset['user_location'] = User_df['user_location']

In [10]:
'''
Sentiment Analysis
-1 (-ve) -> 1 (+ve)
'''
from textblob import TextBlob
import re

sentiment_df = {"sentiment":[]}

for index, row in dataset.iterrows():
  tweet = row['tweet']
  text =  ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", str(tweet)).split())
  polarity = TextBlob(text).sentiment.polarity
  sentiment_df["sentiment"].append(polarity)

sentiment_df = pd.DataFrame(sentiment_df)
dataset = pd.concat([dataset, sentiment_df], axis=1, sort=False)
dataset.head()

,id,conversation_id,created_at,date,timezone,tweet,hashtags,user_id,user_id_str,username,name,day,hour,link,retweet,nlikes,nreplies,nretweets,quote_url,reply_to,plabel,ilabel,followers,following,bio,verified,age,user_location,sentiment
0,1.235145e+18,1.235145e+18,1583316412000,2020-03-04 10:06:52,UTC,#DelhiViolance: Cops arrest man seen firing in...,"['#delhiviolance', '#jaffrabad', '#savedelhihi...",1.057575e+18,1.057575e+18,news_khulasa,Khulasa News,6,10,https://twitter.com/news_khulasa/status/123514...,False,1,0,0,NaN,"[{'user_id': '1057575335280046080', 'username'...",report,fact,1517.0,1491.0,Official Twitter account of khulasa-news. We p...,0.0,506.0,,-0.05
1,1.234889e+18,1.234747e+18,1583255487000,2020-03-03 17:11:27,UTC,Police have asked people to don't come but the...,[],2.159009e+08,2.159009e+08,annuu143,annu kumar singh,1,6,https://twitter.com/annuu143/status/1234889136...,False,0,4,0,NaN,"[{'user_id': '215900941', 'username': 'annuu14...",other,chaos,10.0,55.0,,0.0,3414.0,,0.25
2,1.234812e+18,1.234812e+18,1583237170000,2020-03-03 12:06:10,UTC,"Mohammed Shahrukh, the man who aimed a gun at ...","['#delhiviolence', '#uttarpradesh']",2.264849e+09,2.264849e+09,LogicalIndians,The Logical Indian,6,6,https://twitter.com/LogicalIndians/status/1234...,False,15,2,2,NaN,"[{'user_id': '2264849412', 'username': 'Logica...",report,fact,81439.0,1967.0,"Handpicked, newsworthy stories which deserve t...",1.0,2275.0,India,0.00
3,1.234798e+18,1.234798e+18,1583233754000,2020-03-03 11:09:14,UTC,Delhi shooter Shahrukh (not @iamsrk )who point...,[],7.502518e+07,7.502518e+07,Mohit4Nation,Mohit Saxena,1,17,https://twitter.com/Mohit4Nation/status/123479...,False,0,0,0,NaN,"[{'user_id': '75025176', 'username': 'Mohit4Na...",praise,fact,104.0,98.0,NATIONALIST| STRATEGIST| FOLLOWS THE PRINCIPLE...,0.0,3837.0,India,0.00
4,1.234739e+18,1.234739e+18,1583219760000,2020-03-03 7:16:00,UTC,Ab iski tabiyat se kutai karein @DelhiPolice \...,[],7.815867e+17,7.815867e+17,kalpana0402,🕉 𝕮𝖔𝖚𝖈𝖍 𝕻𝖔𝖙𝖆𝖙𝖔 🇮🇳,7,18,https://twitter.com/kalpana0402/status/1234739...,False,1,0,0,NaN,"[{'user_id': '781586734823845892', 'username':...",action,fact,480.0,352.0,🇮🇳Nationalist|Hindu|Bibliophile/Melophile| Pur...,0.0,1268.0,,0.00


In [0]:
dataset.to_csv('preprocessedDB.csv')